# Tiny-TLS 1.3 Toy Implementation for COSI107a, Spring 2025

This document contains the implementation of a toy version of TLS 1.3, to be used as material for Brandeis' COSI107a course. The goal is to implement a minimalist version of TLS 1.3 that can communicate with a server using the protocol.

This is a work in progress, and it is expected that changes will be made to the protocol as we move forward


## 1. Import the necessary libraries ✅

Since we're working with TLS in Python, it is helpful to use Python libraries that allow us to pack our data into binary form. TLS protocol specifies exact byte length and format, and we'll be doing a lot of conversion between numbers and bytes. We'll use the Python Struct library for this purpose

We'll also be importing the x25519 curve from the Cryptography library to generate the key used in our messages



In [786]:
import struct

from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf import hkdf
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.backends import default_backend

import socket
import time
from urllib.parse import urlparse


import os #for random nonce generation

# 2a. Define some helper functions 🟨

TLS represents all of its messages in byte form. As such, some elements (such as the content length) must be converted from integer to big edian, specifically with 2 bytes. We also need to be able to join the distinct elements in a message into a single byte slice


In [787]:
def u16_to_byte(x: int) -> bytes:
    return struct.pack('>H', x) # Use the struct package to pack a number into big-edian 2 bytes

def concatenate(*bufs: bytes) -> bytes: # Concatenate multiple byte slices into one singular byte slice
    return b''.join(bufs)

def print_bytes_as_hex(b: bytes) -> None: # This is to print the bytes as hex strings. Easier to double check this way
    hex_string = ' '.join([f'{x:02x}' for x in b])
    print(hex_string)

# 2b. Define our private key and public key ✅

We can then generate the public key and private key using the x25519 curve from Cryptography


In [788]:
def key_pair() -> bytes:
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

private_key, public_key = key_pair()

print(private_key.private_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PrivateFormat.Raw,
    encryption_algorithm=serialization.NoEncryption()
    ))
print(public_key.public_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PublicFormat.Raw
    )) 


b'\x08QV\xa0$\xb4i\xfa\x96\xd2\n\n\xb6</\xfa\x83\xb6k;\xaf\xe9\x16\x99\xdc\x8e\x9fek\x16\xf2x'
b' \r\xf7o\xc7.\xbd\x94\xccn%\xcd6&\xe0\xf0\x96K\xd0{\xe3:L\x99[A!Q\xf6!7\x10'


Notice how each time we run the code, the public_key is different. This is intended behavior, as it prevents attackers from being able to predict our key pairs

# 3. The Extension Blueprint: 🟨

Extensions play a large part in shaping a TLS message. From negotiating supported key group to exchanging keys, all of these are achieved using extensions. Luckily for us, these different extensions have a common blueprint



In [789]:
def extension(id: int, content: bytes) -> bytes:
    return concatenate(
        u16_to_byte(id),                         #The ID of the extension. (e.g 0x0a = Supported Group)
        u16_to_byte(len(content)),               #Length of content
        content                                  #The actual content itself
    )

print_bytes_as_hex(extension(0x0a, bytes([0x00, 0x1d]))) #Example Supported Group extension


00 0a 00 02 00 1d


# 4. The ClientHello 🟨

With those building blocks, we can now write our first TLS message: The ClientHello. The ClientHello is always the first message to be sent in a TLS handshake, indicating that the client wants to connect with the server

The ClientHello has these components, in this order:
1. ProtocolVersion (Negotiate which version of TLS we're using)
2. Random Nonce (32 bit, for key generation)
3. Legacy Session ID (For our purposes, we won't be using sessions)
4. Cipher Suites (contains a suite of cipher - how to actually encrypt the key once we have it)
5. Legacy Compression Method (For TLS 1.3, this is null)
6. Extensions 

The code for it is as follows:

In [790]:
def client_hello() -> bytes:
    client_random = os.urandom(32)
    def key_share(pubkey: bytes) -> bytes:      # Encode our public key to be sent over the message
        return concatenate(
            u16_to_byte(len(pubkey) + 4),       # +4 represents the 4 extra byte before the pubkey (2 bytes for the x25519, 2 bytes for len of pubkey)
            u16_to_byte(0x1d),                  # 0x1d is the value for x25519 key
            u16_to_byte(len(pubkey)),           
            pubkey
        )
    
    def DNI(domain: str) -> bytes:
        return concatenate(
            u16_to_byte(len(bytes(domain,'utf-8')) + 3),
            bytes([0x00]),
            u16_to_byte(len(bytes(domain,'utf-8'))),
            bytes(domain, 'utf-8')
        )
    
    def extensions() -> bytes: #This intializes the extensions we need in our message
        return concatenate(
            extension(0x00, DNI('www.cloudflare.com')),
            extension(0x0a, bytes([0x00, 0x02, 0x00, 0x1d])), #Supported Group extensions. Currently only contains the x25519 curve
            extension(0x0d, bytes([0x00, 0x12, 0x04, 0x03, 0x08, 0x04, 0x04, 0x01, 0x05, 0x03, 0x08, 0x05, 0x05, 0x01, 0x08, 0x06, 0x06, 0x01, 0x02, 0x01])),
            extension(0x33, key_share(public_key.public_bytes(
                                encoding=serialization.Encoding.Raw,
                                format=serialization.PublicFormat.Raw
                            ))), #Key Share. Contains the public key generated from the x25519 curve
            extension(0x2b, bytes([0x02, 0x03, 0x04])) #TLS Version. This is how we negotiate TLS 1.3
        )
    
    def handshake() -> bytes: #This constitutes our actual ClientHello message
        return concatenate(
            bytes([0x03, 0x03]),                          # This value is for TLS 1.2. TLS 1.3 must disguise itself as TLS 1.2 to be received, after which it negotiates into TLS 1.3 through the TLS 1.3 extension
            client_random,                             # Random Nonce for key
            bytes([0x00]),                                # Session ID. Empty for our purposes
            bytes([0x00, 0x02, 0x13, 0x02]),              # Cipher Suite. We have a single cipher for our cipher suite (SHA384)
                                                          # I'm aware that there are 2 SHA384 ciphers: AES and CHACHA. I've included one here. Not sure
                                                          # if we need the other one or not
            bytes([0x01, 0x00]),                          # Compression Method. Empty for our purposes
            u16_to_byte(len(extensions())),
            extensions()
        )
    
    return concatenate(                                 #Include record layers for TLS 1.3 to complete message
        bytes([0x16, 0x03, 0x01]),
        u16_to_byte(len(handshake()) + 4),
        bytes([0x01,0x00]),
        u16_to_byte(len(handshake())),
        handshake()
    ), client_random

client_hello_msg, client_random = client_hello()
print_bytes_as_hex(client_hello_msg)
client_hello_no_record = client_hello_msg[5:] #This is to calculate the transcript hash
print_bytes_as_hex(client_hello_no_record)


## To check the validity of the ClientHello, copy paste the ClientHello into the openSSL_debug.py script and run it. Server should give back a response

16 03 01 00 9b 01 00 00 97 03 03 ae a2 33 02 24 f0 63 33 6d 3f 0d 6f e4 cc 45 12 77 10 96 85 ab 47 f0 e4 93 8f bc 30 45 dc 08 d4 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 20 0d f7 6f c7 2e bd 94 cc 6e 25 cd 36 26 e0 f0 96 4b d0 7b e3 3a 4c 99 5b 41 21 51 f6 21 37 10 00 2b 00 03 02 03 04
01 00 00 97 03 03 ae a2 33 02 24 f0 63 33 6d 3f 0d 6f e4 cc 45 12 77 10 96 85 ab 47 f0 e4 93 8f bc 30 45 dc 08 d4 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 20 0d f7 6f c7 2e bd 94 cc 6e 25 cd 36 26 e0 f0 96 4b d0 7b e3 3a 4c 99 5b 41 21 51 f6 21 37 10 00 2b 00 03 02 03 04


# 5. Sending the ClientHello: 🟨
Since we now have the ClientHello, the next step would be to send it over to a public domain and see if the server responds. For this part, we'll be using www.cloudbase.com, as it supports TLS 1.3, and the cipher suite matches with our clientHello.


In [791]:
server_hello_no_record = b""
encrypted_payload = b""



def parse_tls_record(data):
    #Parse a TLS record and return its components
    if len(data) < 5:  # Minimum TLS record length
        return None, None, None, data
    
    content_type = data[0] #Parse the record header to see which type of record this is.
    if (data[1] == 3) and (data[2] == 3):
        version = "1.2/1.3"
    length = int.from_bytes(data[3:5], byteorder='big')
    
    if len(data) < length + 5:
        return None, None, None, data
        
    if content_type == 0x17:
        payload = data[:5+length]  # Include the header for application data. We need the header for the decryption algorithm
        remaining_data = data[5+length:]
    else:
        payload = data[5:5+length]  # Original behavior for other types
        remaining_data = data[5+length:]
    
    content_type_names = { #Dictionary indicating the types of TLS records we may receive
        0x14: "Change Cipher Spec",
        0x15: "Alert",
        0x16: "Handshake",
        0x17: "Application Data"
    }
    
    type_name = content_type_names.get(content_type, f"Unknown ({content_type})")
    
    return type_name, version, payload, remaining_data

def parse_handshake_message(data): #This is exclusive for Handshake messages. We'll primarily use this to identify serverHellos.
    """Parse a TLS handshake message."""
    if len(data) < 4:
        return None, None, None
        
    msg_type = data[0]
    length = int.from_bytes(data[1:4], byteorder='big')
    
    handshake_types = {
        0x00: "Hello Request",
        0x01: "Client Hello",
        0x02: "Server Hello",
        0x0b: "Certificate",
        0x0c: "Server Key Exchange",
        0x0d: "Certificate Request",
        0x0e: "Server Hello Done",
        0x0f: "Certificate Verify",
        0x10: "Client Key Exchange",
        0x14: "Finished",
        0x08: "Encrypted Extensions",  # Added TLS 1.3 message types
        0x13: "Certificate Status"
    }
    
    type_name = handshake_types.get(msg_type, f"Unknown ({msg_type})")
    if len(data) >= 4+length:
        payload = data[:4+length]
    
    return type_name, length, payload

def format_bytes(data):
    """Format bytes as a hexadecimal string with ASCII representation."""
    hex_dump = ' '.join(f'{b:02x}' for b in data)
    return f"Hex: {hex_dump}"

def receive_all_data(sock, timeout=5):
    """Receive all data from the socket until timeout or connection close."""
    buffer = b""
    sock.settimeout(timeout)
    start_time = time.time()
    
    while True:
        try:
            chunk = sock.recv(16384)
            if not chunk:  # Connection closed by server
                break
            buffer += chunk
            
            # Check timeout
            if time.time() - start_time > timeout:
                break
                
        except socket.timeout:
            break
            
    return buffer
# Parse the URL to get the hostname
parsed_url = urlparse('https://www.cloudflare.com/')
host = parsed_url.hostname
port = 443
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
print(f"Attempting to connect to {host}:{port}")
s.connect((host, port))
print("Connected to server.")

# Your existing client_hello
try:
    
    bytes_sent = s.sendall(client_hello_msg)
    print(f"Sent ClientHello ({len(client_hello_msg)} bytes)")
    
    print("\nWaiting for server response...")
    try:
        buffer = receive_all_data(s)
        print(f"\nReceived {len(buffer)} new bytes")
        print(print_bytes_as_hex(buffer))


        
        # Process all complete TLS records in the buffer
        while buffer:
            record_type, version, payload, remaining = parse_tls_record(buffer)
            if record_type is None:  # Incomplete record
                break
                
            buffer = remaining
            print("\n" + "="*50)
            print(f"Record Type: {record_type}")
            print(f"TLS Version: {version}")
            
            if record_type == "Handshake":
                msg_type, msg_length, msg_payload = parse_handshake_message(payload)
                if msg_type:
                    print(f"Handshake Type: {msg_type}")
                    print(f"Message Length: {msg_length}")
                    if msg_payload:
                        print("\nMessage Payload:")
                        print(format_bytes(msg_payload))
                        server_hello_no_record = msg_payload
            else:
                print("\nRecord Payload:")
                if record_type == "Application Data":
                    encrypted_payload = payload
                print(format_bytes(payload))
            
    except socket.timeout:
        print("Timeout while waiting for server response")
            
except ConnectionRefusedError:
    print("Connection refused. Is the OpenSSL server running?")
except socket.timeout:
    print("Connection attempt timed out")
except Exception as e:
    print(f"Error: {e}")

print("\nExtracted Data:")
if server_hello_no_record:
    print("\nServer Hello (without record header):")
    print(print_bytes_as_hex(server_hello_no_record))
if encrypted_payload:
    print("\nEncrypted Payload (with record header):")
    print(print_bytes_as_hex(encrypted_payload))

Attempting to connect to www.cloudflare.com:443
Connected to server.
Sent ClientHello (160 bytes)

Waiting for server response...

Received 2820 new bytes
16 03 03 00 5a 02 00 00 56 03 03 06 f3 47 79 8d 67 a4 67 cd a1 ba 0c c1 5b eb 3e 91 31 71 9e 5f e9 89 de 84 55 04 a4 b2 bb 33 80 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 48 fa 2d d4 05 e0 7d 21 2a cb f2 8f 08 8a 66 36 a7 19 38 75 ec 04 f3 c4 f7 4a 47 8a 4c ea 7e 5a 00 2b 00 02 03 04 14 03 03 00 01 01 17 03 03 0a 9a ff 5f 8a 9f 7b 88 f3 47 da 11 32 1c 2b cc fa 8a fb a7 02 2d c7 c6 ac a3 e7 94 df bf 9b 5b f2 04 f3 ae 79 f8 51 dc f0 47 f9 ce 81 5f 07 bf db b8 d3 b2 2b f5 b3 fc d3 fc 11 3f 56 10 7d 12 ba 94 15 fe 51 b7 46 f0 63 ff 39 00 8b 70 db ab 04 93 bd 9a 90 77 cf e7 1d 41 50 ce e1 62 22 14 1c e3 02 a4 4b 10 8b 21 3e 37 3d 44 bf 8a 94 63 02 83 4c a9 95 ba 2f 82 2b 78 4f da 24 23 03 9e 47 55 f3 21 24 16 cf b0 7d 15 e0 59 d6 de 50 fb d5 9d e7 6f 4a bc ad 4c 8a 44 0e f5 1b b9 52 0c 19 b5 b6 1e 8d 49 fe fd 51 62 35 5d b6 e5 1d ea f6 73

# 6. Parsing the ServerHello ✅

Once the ClientHello is sent, the server responds with its own message. This message contains the ServerHello, the CipherChangeSpec, and the encrypted payload. Assuming that our ClientHello message is configured correctly, the server will respond with its chosen cipher suite and its own key. 

We are particularly interested in the server's key and server random, which we use to establish cryptographic parameters. We can then extract these properties and start the key calculation process

# 6a. Creating a Parser class ✅

The ServerHello message isn't always of the same size. Certain elements like the sessionID, as well as the content of extensions, may have variable length depending on the message itself. That's why we can't just extract the information based on indexes alone. We need a parser that would keep track of what element we're at and how many bytes we have to skip forward


In [792]:

class Parser:
    def __init__(self, data: bytes) -> None:
        self.data = data
        self.cursor = 0
    
    def skip(self, position: int) -> None:
        self.cursor += position

    def read(self, position: int) -> bytes:
        result = self.data[self.cursor : self.cursor + position]
        self.cursor += position
        return result 
    
    def read_uint8_prefixed(self) -> bytes:                           #Most extensions with variable lengths have bytes that denote their length. We can use this to skip forward appropriately 
        length = self.data[self.cursor]
        print(length)
        self.cursor += 1
        result = self.data[self.cursor : self.cursor + length]
        self.cursor += length
        return result

    def read_uint16_prefixed(self) -> bytes:
        length = int.from_bytes(self.data[self.cursor:self.cursor + 2], 'big') #Since some lengths are represented with 2 bytes, we need to convert them 
        self.cursor += 2
        result = self.data[self.cursor:self.cursor + length]
        self.cursor += length
        return result


# 6b. Parsing the ServerHello ✅

With our Parser class, we can now parse the ServerHello to extract the ServerRandom and the public key. Keep in mind that we are only dealing with 1 cipher suite, 1 cryptographic and as such expects only 1 public key. A full version of TLS 1.3 will be much more complex

In [793]:
def server_hello_parser(msg: bytes):
    parser = Parser(msg)
    parser.skip(4)                                  #Skip HandShake Header
    parser.skip(2)                                  #Skip Server Version
    server_random = parser.read(32)                 #Get the serverRandom
    parser.read_uint8_prefixed()                    #Skip SessionID
    parser.skip(2)                                  #Skip Cipher Suite
    parser.skip(1)                                  #Skip Compression Method
    public_key = None
    extensions = parser.read_uint16_prefixed()
    extension_reader = Parser(extensions)
    while(extension_reader.cursor < len(extensions)):
        extension_type = extension_reader.read(2)
        extension_data = extension_reader.read_uint16_prefixed()
        if (extension_type == b'\x00\x33'):
            data = Parser(extension_data)
            data.skip(2)
            public_key = data.read_uint16_prefixed()
    return server_random, public_key


#Pre-generated server_hello_msg, used for debugging purposes
#server_hello_no_record = bytes.fromhex("02 00 00 76 03 03 70 71 72 73 74 75 76 77 78 79 7a 7b 7c 7d 7e 7f 80 81 82 83 84 85 86 87 88 89 8a 8b 8c 8d 8e 8f 20 e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff 13 02 00 00 2e 00 2b 00 02 03 04 00 33 00 24 00 1d 00 20 9f d7 ad 6d cf f4 29 8d d3 f9 6d 5b 1b 2a f9 10 a0 53 5b 14 88 d7 f8 fa bb 34 9a 98 28 80 b6 15")
#Run the script with the pre-generated server hello and compare with https://tls13.xargs.org/#server-handshake-keys-calc to see if the script is working
#Current serverHello generated from www.cloudbase.com. Right now I'm copy-pasting from the terminal when I run the debug.py script. Also parser is ignoring the record header.
server_random, public_key = server_hello_parser(server_hello_no_record)
print_bytes_as_hex(server_random)
print_bytes_as_hex(public_key)


           

        
        
        


0
06 f3 47 79 8d 67 a4 67 cd a1 ba 0c c1 5b eb 3e 91 31 71 9e 5f e9 89 de 84 55 04 a4 b2 bb 33 80
48 fa 2d d4 05 e0 7d 21 2a cb f2 8f 08 8a 66 36 a7 19 38 75 ec 04 f3 c4 f7 4a 47 8a 4c ea 7e 5a


# 7. Key Derivation ✅

Now that we have our private key and the server has sent back their public key, we can now encrypt our data to the server via key calculations. We can break down the key calculations into a few steps

# 7a. Transcript Hash ✅

Transcript Hash refers to the hash of the ClientHello and the ServerHello messages. The idea is to associate these messages with the keys we are about to derive: These keys work with these messages. That way, even if an attacker somehow got hold of our keys, it would not work with whatever messages they try to send



In [794]:
def transcript_hash(client: bytes, server: bytes) -> bytes:
    digest = hashes.Hash(hashes.SHA384()) # We'll be using SHA384 to create the hash for our messages.
    digest.update(client)     # Note that both the ClientHello and ServerHello is used here
    digest.update(server)
    transcript_hash = digest.finalize()
    return transcript_hash

print_bytes_as_hex(client_hello_no_record)       #This is the clientHello message we generated from our code
print_bytes_as_hex(server_hello_no_record)       #Right now, this server_hello is pre-generated for testing purposes. We'll be parsing an actual server hello when the entire thing is ready

# Testing value for transcript hash calculations
# client_hello_no_record = bytes.fromhex("01 00 00 f4 03 03 00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 20 e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff 00 08 13 02 13 03 13 01 00 ff 01 00 00 a3 00 00 00 18 00 16 00 00 13 65 78 61 6d 70 6c 65 2e 75 6c 66 68 65 69 6d 2e 6e 65 74 00 0b 00 04 03 00 01 02 00 0a 00 16 00 14 00 1d 00 17 00 1e 00 19 00 18 01 00 01 01 01 02 01 03 01 04 00 23 00 00 00 16 00 00 00 17 00 00 00 0d 00 1e 00 1c 04 03 05 03 06 03 08 07 08 08 08 09 08 0a 08 0b 08 04 08 05 08 06 04 01 05 01 06 01 00 2b 00 03 02 03 04 00 2d 00 02 01 01 00 33 00 26 00 24 00 1d 00 20 35 80 72 d6 36 58 80 d1 ae ea 32 9a df 91 21 38 38 51 ed 21 a2 8e 3b 75 e9 65 d0 d2 cd 16 62 54")
# server_hello_no_record = bytes.fromhex("02 00 00 76 03 03 70 71 72 73 74 75 76 77 78 79 7a 7b 7c 7d 7e 7f 80 81 82 83 84 85 86 87 88 89 8a 8b 8c 8d 8e 8f 20 e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff 13 02 00 00 2e 00 2b 00 02 03 04 00 33 00 24 00 1d 00 20 9f d7 ad 6d cf f4 29 8d d3 f9 6d 5b 1b 2a f9 10 a0 53 5b 14 88 d7 f8 fa bb 34 9a 98 28 80 b6 15")


hello_hash = transcript_hash(client_hello_no_record, server_hello_no_record)
print_bytes_as_hex(hello_hash)

01 00 00 97 03 03 ae a2 33 02 24 f0 63 33 6d 3f 0d 6f e4 cc 45 12 77 10 96 85 ab 47 f0 e4 93 8f bc 30 45 dc 08 d4 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 20 0d f7 6f c7 2e bd 94 cc 6e 25 cd 36 26 e0 f0 96 4b d0 7b e3 3a 4c 99 5b 41 21 51 f6 21 37 10 00 2b 00 03 02 03 04
02 00 00 56 03 03 06 f3 47 79 8d 67 a4 67 cd a1 ba 0c c1 5b eb 3e 91 31 71 9e 5f e9 89 de 84 55 04 a4 b2 bb 33 80 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 48 fa 2d d4 05 e0 7d 21 2a cb f2 8f 08 8a 66 36 a7 19 38 75 ec 04 f3 c4 f7 4a 47 8a 4c ea 7e 5a 00 2b 00 02 03 04
96 46 df 11 96 6b 43 e3 af 53 ca af 9e 74 9c cf 22 39 9a ef d7 5e 3c 28 e4 dd 51 1f 2d 06 74 2c d5 9b 50 cb 3b 58 bb b8 8f a8 fa d0 68 6b 53 71


# 7b. Shared secret ✅
The idea of the key exchange is that given the private key and the public key of the other party, both client and server can perform calculations to arrive at the same number. Essentially, client private x server public = server private x client public. This is called the shared secret. We'll perform the calculations on our side while the server does it on theirs.


In [795]:
# Our public key right now is raw bytes. Our private key, on the other hand, is a x25519PrivateKey object. The most convenient way to calculate the 
# shared secret is to turn the public key into a x25519PublicKey object and leverage the built in methods to generate the shared secret


#Test key exchange
# test_client_private_key = x25519.X25519PrivateKey.from_private_bytes(bytes.fromhex("202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f"))
# test_client_public_key = test_client_private_key.public_key()
# print_bytes_as_hex(test_client_public_key.public_bytes(
#     encoding=serialization.Encoding.Raw,
#     format=serialization.PublicFormat.Raw
#     ))

# test_server_private_key = x25519.X25519PrivateKey.from_private_bytes(bytes.fromhex("909192939495969798999a9b9c9d9e9fa0a1a2a3a4a5a6a7a8a9aaabacadaeaf"))
# test_server_public_key = test_server_private_key.public_key()
# print_bytes_as_hex(test_server_public_key.public_bytes(
#     encoding=serialization.Encoding.Raw,
#     format=serialization.PublicFormat.Raw
#     ))

# test_shared_secret = test_client_private_key.exchange(test_server_public_key)
# print_bytes_as_hex(test_shared_secret)


server_public_key = x25519.X25519PublicKey.from_public_bytes(public_key) 
# private_key = x25519.X25519PrivateKey.from_private_bytes(bytes.fromhex("202122232425262728292a2b2c2d2e2f303132333435363738393a3b3c3d3e3f"))
shared_secret = private_key.exchange(server_public_key)
print_bytes_as_hex(shared_secret)







17 a0 6c 22 ad 3b b7 4e c1 19 02 79 5a 0b 33 1e b3 af 4e 31 b1 bb 16 57 70 b5 72 6b a0 6f 45 6c


# 7c. Early Secret, Derived Secret, Handshake Secret ✅
We then go on to generate the early secret, the derived secret and finally the handshake secret. 

- Early secret is for 0-RTT data (essentially data sent with pre-shared keys). Since we're not using PSKs, this is a bunch of 0 bytes.
- Derived secret is mainly used as salt, mixed in with the early secret to get the handshake key. This way, even if an attacker has the early secret, it's hard to determine the handshake key

Each of these secrets have a "label" associated with them to identify their use, usually with the format "tls13 + ID of the component". 

* Note 1: We can pass in the label directly into the HKDF info parameter, or we can make a method that will append the TLS13 infront and add the necessary bytes for length, after which it is passed into the infor parameter. To avoid hardcoding, I went with the second options. If this example works, we can go back and try hardcoding the labels to see if it affects anything


In [796]:
## REMEMBER TO CHANGE IT BACK TO SHA256 AFTER TESTING. Algorithm goes back to SHA256, Length goes back to 32.


# UNCOMMENT THIS FOR TESTING. SHOULD WORK FOR SHA384 IMPLEMENTATIONS:

# shared_secret = bytes.fromhex('df4a291baa1eb7cfa6934b29b474baad2697e29f1f920dcc77c8a0a088447624')
# hello_hash = bytes.fromhex('e05f64fcd082bdb0dce473adf669c2769f257a1c75a51b7887468b5e0e7a7de4f4d34555112077f16e079019d5a845bd')




# TLS specs make use of 2 methods: HKDF-Extract and HKDF-Expand. HKDF-Extract can be found in hkdf.HKDF()_extract(). HKDF-Expand is hkdf.HKDFExpand()
# Define our own hkdf_expand_label. This is basically calling HKDF to derive secrets, with an added step of configuring the labels associated with the component.

def hkdf_expand_label(secret: bytes, label: str, context: bytes, length: int) -> bytes:
    # Construct the HkdfLabel as specified in TLS 1.3
    label = b"tls13 " + label.encode('utf-8')
    hkdf_label = (
        struct.pack("!H", length) +  # 2 bytes for length
        struct.pack("!B", len(label)) + label +  # 1 byte for label length + label
        struct.pack("!B", len(context)) + context  # 1 byte for context length + context
    )
    
    # Use HKDF-Expand
    encryption = hkdf.HKDFExpand(
        algorithm=hashes.SHA384(),
        length=length,
        info=hkdf_label,
    )
    
    return encryption.derive(secret)


early_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(), #We negotiated SHA256 in our cipher suite
    length=48,
    salt=b'\x00',
    info=b'\x00'
)._extract(b'\x00' * 48) #The first input key is simply 00

empty_hash = hashes.Hash(hashes.SHA384())
empty_hash.update(b"")  # Empty string
empty_hash = empty_hash.finalize()

# We then derive another secret called the derived secret from the early secret
derived_secret = hkdf_expand_label(
    secret = early_secret,
    label="derived",
    context=empty_hash,
    length=48
)

# The derived secret is mixed into the early secret to get the handshake secret
handshake_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(),
    length=48,
    salt=derived_secret,  # derived_secret is used as salt
    info=None
)._extract(shared_secret)

# 7d. Traffic Secrets and Key derivation ✅

Now that we have our handshake secret, we can now move on to derive the traffic secrets, and subsequently the keys and IVs for both the client and the server

In [797]:
# Deriving the secrets and subsequent keys + IVs.
client_secret = hkdf_expand_label(
    secret = handshake_secret, 
    label = "c hs traffic", 
    context = hello_hash, 
    length = 48
)

server_secret = hkdf_expand_label(
    secret = handshake_secret,
    label = "s hs traffic",
    context = hello_hash,
    length = 48
)


client_handshake_key = hkdf_expand_label(
    secret = client_secret,
    label = "key",
    context = b"",
    length = 32
)

server_handshake_key = hkdf_expand_label(
    secret = server_secret,
    label = "key",
    context = b"",
    length = 32
)

client_handshake_iv = hkdf_expand_label(
    secret = client_secret,
    label = "iv",
    context = b"",
    length = 12
)

server_handshake_iv = hkdf_expand_label(
    secret = server_secret,
    label = "iv",
    context = b"",
    length = 12
)




print("handshake secret: " + handshake_secret.hex())
print("client secret: " + client_secret.hex())
print("server secret: " + server_secret.hex())
print("client handshake key: " + client_handshake_key.hex())
print("client handshake iv: "+ client_handshake_iv.hex())
print("server handshake key: " + server_handshake_key.hex())
print("server handshake iv: " + server_handshake_iv.hex())


handshake secret: 9ff665d98d7c184011f1d1e55e1340cf82f10a7600db3815115ebc19c35bc4e70de33ca19dc3696b6240494cdc6182d2
client secret: a3c5b66a2198943058e9062fc479e09f03248f838f31bbc866c5090a77da01c8a0d4ede4bb1e566df75d40ed0ef02761
server secret: f55cacb1e93b11d5fc127f8783db01c79d076c4930a70828881ee26e3ab031fde9cd2f5304137a8fb56533bb7738e0e6
client handshake key: 5d1e7bb231625c53bcbe3885d6d90e2dc3b13dc71afd87200cf905d9ef8430b3
client handshake iv: d0c2801f189047beabe7a538
server handshake key: 53d93a4c7f162ee1b3779de5fb4e46ca001ae4013706658f3b3440965de9db83
server handshake iv: f1f3cc4b43824bf345cd4382


## 8. Decryption ✅

Now that we have the keys and IVs, the next step would be to decrypt the encrypted records. When sending the ClientHello, the server sends back a massive binary dump, including the ServerHello, the change cipher specs, and the actual payload, which is encrypted. We will use the keys and ivs generated to decrypt these records.

Normally, you'll see a bunch of records, each with its own content, from the server encrypted extension, certificates, etc. However, Cloudbase has aggregated all of these into a single record. What we're really interested in is the server's finished message.

In [798]:
def decrypt(key, iv, wrapper):
    # Split the wrapper into additional authenticated data (AAD) and ciphertext
    additional = wrapper[:5]
    ciphertext = wrapper[5:]
    
    # Initialize AES-GCM with the provided key
    aesgcm = AESGCM(key)
    
    # Decrypt the ciphertext
    try:
        plaintext = aesgcm.decrypt(iv, ciphertext, additional)
    except Exception as e:
        raise ValueError(f"Decryption failed: {e}")
    
    return plaintext



# Right now I'm doing a lot of copy pasting. I run the openSSL_debug script to retrieve the server's response, and then replace the serverHello, as well as the encrypted record under this line. Managed to decrypt the thing.

payload = decrypt(server_handshake_key, server_handshake_iv, encrypted_payload)[:-1]
print_bytes_as_hex(payload)

#Test payload:
#server_handshake_key = bytes.fromhex("9f13575ce3f8cfc1df64a77ceaffe89700b492ad31b4fab01c4792be1b266b7f")
#server_handshake_iv = bytes.fromhex("9563bc8b590f671f488d2da3")
#encrypted_payload = bytes.fromhex("17 03 03 00 17 6b e0 2f 9d a7 c2 dc 9d de f5 6f 24 68 b9 0a df a2 51 01 ab 03 44 ae")
#payload = decrypt(server_handshake_key, server_handshake_iv, encrypted_payload)
#print("Test payload: " + payload.hex())





08 00 00 06 00 04 00 00 00 00 0b 00 09 f8 00 00 09 f4 00 03 c4 30 82 03 c0 30 82 03 66 a0 03 02 01 02 02 10 12 c6 55 b2 e7 0b ce 7c 13 1a 7c e5 a2 e0 e7 00 30 0a 06 08 2a 86 48 ce 3d 04 03 02 30 3b 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 1e 30 1c 06 03 55 04 0a 13 15 47 6f 6f 67 6c 65 20 54 72 75 73 74 20 53 65 72 76 69 63 65 73 31 0c 30 0a 06 03 55 04 03 13 03 57 45 31 30 1e 17 0d 32 35 30 33 30 31 31 37 34 34 32 30 5a 17 0d 32 35 30 35 33 30 31 38 34 34 31 38 5a 30 1d 31 1b 30 19 06 03 55 04 03 13 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 30 59 30 13 06 07 2a 86 48 ce 3d 02 01 06 08 2a 86 48 ce 3d 03 01 07 03 42 00 04 50 1f b6 ee 9c c2 99 12 65 31 48 92 cb ec fa 69 ea d3 d4 4c 52 03 d6 cd d9 60 3d 2a 1c b0 04 a0 84 ff 88 84 fc dd 83 41 74 5e 1c ad 8b bd 2f e3 10 30 44 25 30 e3 f6 23 af 73 c6 2e 82 9f 33 5c a3 82 02 68 30 82 02 64 30 0e 06 03 55 1d 0f 01 01 ff 04 04 03 02 07 80 30 13 06 03 55 1d 25 04 0c 30 0a 06 08 2b 06 01 05 05 07 03 01 30 0c 06 03 55 1d 13 01 01 f

# 9. Application keys: ✅
The application keys are nearly the last step when it comes to finishing the handshake. The handshake keys were there decrypt the server's response, containing encrypted extensions, certificates, etc. Once we decrypt the encryption, we then use them to generate another set of keys, called application keys. This will be the main keys we use to communicate with the server. The code themselves are basically the same as the ones used to generate handshake hash

# 9a. Application Key Hash: ✅
Generate a hash of all the messages up until this point: (Client Hello, Server Hello, [unwrapped] Encrypted Extensions, [unwrapped] Server Certificate, [unwrapped] Server Certificate Verify, [unwrapped] Server Finished). This will be used to generate the application key

In [799]:
## ------- Potential bug in application_key_hash generation ------------

def application_key_hash(client: bytes, server: bytes, payload: bytes) -> bytes:
    digest = hashes.Hash(hashes.SHA384()) # We'll be using SHA384 to create the hash for our messages.
    digest.update(client)     # Note that both the ClientHello and ServerHello is used here
    digest.update(server)
    digest.update(payload)
    application_key_hash = digest.finalize()
    return application_key_hash

print_bytes_as_hex(client_hello_no_record)
print_bytes_as_hex(server_hello_no_record)
print_bytes_as_hex(payload)
application_context = application_key_hash(client_hello_no_record, server_hello_no_record, payload)
print_bytes_as_hex(application_context)




# test_client_hello = bytes.fromhex("01 00 00 f4 03 03 00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f 20 e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff 00 08 13 02 13 03 13 01 00 ff 01 00 00 a3 00 00 00 18 00 16 00 00 13 65 78 61 6d 70 6c 65 2e 75 6c 66 68 65 69 6d 2e 6e 65 74 00 0b 00 04 03 00 01 02 00 0a 00 16 00 14 00 1d 00 17 00 1e 00 19 00 18 01 00 01 01 01 02 01 03 01 04 00 23 00 00 00 16 00 00 00 17 00 00 00 0d 00 1e 00 1c 04 03 05 03 06 03 08 07 08 08 08 09 08 0a 08 0b 08 04 08 05 08 06 04 01 05 01 06 01 00 2b 00 03 02 03 04 00 2d 00 02 01 01 00 33 00 26 00 24 00 1d 00 20 35 80 72 d6 36 58 80 d1 ae ea 32 9a df 91 21 38 38 51 ed 21 a2 8e 3b 75 e9 65 d0 d2 cd 16 62 54")
# test_server_hello = bytes.fromhex("02 00 00 76 03 03 70 71 72 73 74 75 76 77 78 79 7a 7b 7c 7d 7e 7f 80 81 82 83 84 85 86 87 88 89 8a 8b 8c 8d 8e 8f 20 e0 e1 e2 e3 e4 e5 e6 e7 e8 e9 ea eb ec ed ee ef f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 fa fb fc fd fe ff 13 02 00 00 2e 00 2b 00 02 03 04 00 33 00 24 00 1d 00 20 9f d7 ad 6d cf f4 29 8d d3 f9 6d 5b 1b 2a f9 10 a0 53 5b 14 88 d7 f8 fa bb 34 9a 98 28 80 b6 15")
# # test_extension_1 = bytes.fromhex("08 00 00 02 00 00")
# # test_extension_2 = bytes.fromhex("0b 00 03 2e 00 00 03 2a 00 03 25 30 82 03 21 30 82 02 09 a0 03 02 01 02 02 08 15 5a 92 ad c2 04 8f 90 30 0d 06 09 2a 86 48 86 f7 0d 01 01 0b 05 00 30 22 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 13 30 11 06 03 55 04 0a 13 0a 45 78 61 6d 70 6c 65 20 43 41 30 1e 17 0d 31 38 31 30 30 35 30 31 33 38 31 37 5a 17 0d 31 39 31 30 30 35 30 31 33 38 31 37 5a 30 2b 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 1c 30 1a 06 03 55 04 03 13 13 65 78 61 6d 70 6c 65 2e 75 6c 66 68 65 69 6d 2e 6e 65 74 30 82 01 22 30 0d 06 09 2a 86 48 86 f7 0d 01 01 01 05 00 03 82 01 0f 00 30 82 01 0a 02 82 01 01 00 c4 80 36 06 ba e7 47 6b 08 94 04 ec a7 b6 91 04 3f f7 92 bc 19 ee fb 7d 74 d7 a8 0d 00 1e 7b 4b 3a 4a e6 0f e8 c0 71 fc 73 e7 02 4c 0d bc f4 bd d1 1d 39 6b ba 70 46 4a 13 e9 4a f8 3d f3 e1 09 59 54 7b c9 55 fb 41 2d a3 76 52 11 e1 f3 dc 77 6c aa 53 37 6e ca 3a ec be c3 aa b7 3b 31 d5 6c b6 52 9c 80 98 bc c9 e0 28 18 e2 0b f7 f8 a0 3a fd 17 04 50 9e ce 79 bd 9f 39 f1 ea 69 ec 47 97 2e 83 0f b5 ca 95 de 95 a1 e6 04 22 d5 ee be 52 79 54 a1 e7 bf 8a 86 f6 46 6d 0d 9f 16 95 1a 4c f7 a0 46 92 59 5c 13 52 f2 54 9e 5a fb 4e bf d7 7a 37 95 01 44 e4 c0 26 87 4c 65 3e 40 7d 7d 23 07 44 01 f4 84 ff d0 8f 7a 1f a0 52 10 d1 f4 f0 d5 ce 79 70 29 32 e2 ca be 70 1f df ad 6b 4b b7 11 01 f4 4b ad 66 6a 11 13 0f e2 ee 82 9e 4d 02 9d c9 1c dd 67 16 db b9 06 18 86 ed c1 ba 94 21 02 03 01 00 01 a3 52 30 50 30 0e 06 03 55 1d 0f 01 01 ff 04 04 03 02 05 a0 30 1d 06 03 55 1d 25 04 16 30 14 06 08 2b 06 01 05 05 07 03 02 06 08 2b 06 01 05 05 07 03 01 30 1f 06 03 55 1d 23 04 18 30 16 80 14 89 4f de 5b cc 69 e2 52 cf 3e a3 00 df b1 97 b8 1d e1 c1 46 30 0d 06 09 2a 86 48 86 f7 0d 01 01 0b 05 00 03 82 01 01 00 59 16 45 a6 9a 2e 37 79 e4 f6 dd 27 1a ba 1c 0b fd 6c d7 55 99 b5 e7 c3 6e 53 3e ff 36 59 08 43 24 c9 e7 a5 04 07 9d 39 e0 d4 29 87 ff e3 eb dd 09 c1 cf 1d 91 44 55 87 0b 57 1d d1 9b df 1d 24 f8 bb 9a 11 fe 80 fd 59 2b a0 39 8c de 11 e2 65 1e 61 8c e5 98 fa 96 e5 37 2e ef 3d 24 8a fd e1 74 63 eb bf ab b8 e4 d1 ab 50 2a 54 ec 00 64 e9 2f 78 19 66 0d 3f 27 cf 20 9e 66 7f ce 5a e2 e4 ac 99 c7 c9 38 18 f8 b2 51 07 22 df ed 97 f3 2e 3e 93 49 d4 c6 6c 9e a6 39 6d 74 44 62 a0 6b 42 c6 d5 ba 68 8e ac 3a 01 7b dd fc 8e 2c fc ad 27 cb 69 d3 cc dc a2 80 41 44 65 d3 ae 34 8c e0 f3 4a b2 fb 9c 61 83 71 31 2b 19 10 41 64 1c 23 7f 11 a5 d6 5c 84 4f 04 04 84 99 38 71 2b 95 9e d6 85 bc 5c 5d d6 45 ed 19 90 94 73 40 29 26 dc b4 0e 34 69 a1 59 41 e8 e2 cc a8 4b b6 08 46 36 a0 00 00")
# # test_extension_3 = bytes.fromhex("0f 00 01 04 08 04 01 00 5c bb 24 c0 40 93 32 da a9 20 bb ab bd b9 bd 50 17 0b e4 9c fb e0 a4 10 7f ca 6f fb 10 68 e6 5f 96 9e 6d e7 d4 f9 e5 60 38 d6 7c 69 c0 31 40 3a 7a 7c 0b cc 86 83 e6 57 21 a0 c7 2c c6 63 40 19 ad 1d 3a d2 65 a8 12 61 5b a3 63 80 37 20 84 f5 da ec 7e 63 d3 f4 93 3f 27 22 74 19 a6 11 03 46 44 dc db c7 be 3e 74 ff ac 47 3f aa ad de 8c 2f c6 5f 32 65 77 3e 7e 62 de 33 86 1f a7 05 d1 9c 50 6e 89 6c 8d 82 f5 bc f3 5f ec e2 59 b7 15 38 11 5e 9c 8c fb a6 2e 49 bb 84 74 f5 85 87 b1 1b 8a e3 17 c6 33 e9 c7 6c 79 1d 46 62 84 ad 9c 4f f7 35 a6 d2 e9 63 b5 9b bc a4 40 a3 07 09 1a 1b 4e 46 bc c7 a2 f9 fb 2f 1c 89 8e cb 19 91 8b e4 12 1d 7e 8e d0 4c d5 0c 9a 59 e9 87 98 01 07 bb bf 29 9c 23 2e 7f db e1 0a 4c fd ae 5c 89 1c 96 af df f9 4b 54 cc d2 bc 19 d3 cd aa 66 44 85 9c")
# # test_extension_4 = bytes.fromhex("14 00 00 30 7e 30 ee cc b6 b2 3b e6 c6 ca 36 39 92 e8 42 da 87 7e e6 47 15 ae 7f c0 cf 87 f9 e5 03 21 82 b5 bb 48 d1 e3 3f 99 79 05 5a 16 0c 8d bb b1 56 9c")
# test_extension_all = bytes.fromhex("08 00 00 02 00 00 0b 00 03 2e 00 00 03 2a 00 03 25 30 82 03 21 30 82 02 09 a0 03 02 01 02 02 08 15 5a 92 ad c2 04 8f 90 30 0d 06 09 2a 86 48 86 f7 0d 01 01 0b 05 00 30 22 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 13 30 11 06 03 55 04 0a 13 0a 45 78 61 6d 70 6c 65 20 43 41 30 1e 17 0d 31 38 31 30 30 35 30 31 33 38 31 37 5a 17 0d 31 39 31 30 30 35 30 31 33 38 31 37 5a 30 2b 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 1c 30 1a 06 03 55 04 03 13 13 65 78 61 6d 70 6c 65 2e 75 6c 66 68 65 69 6d 2e 6e 65 74 30 82 01 22 30 0d 06 09 2a 86 48 86 f7 0d 01 01 01 05 00 03 82 01 0f 00 30 82 01 0a 02 82 01 01 00 c4 80 36 06 ba e7 47 6b 08 94 04 ec a7 b6 91 04 3f f7 92 bc 19 ee fb 7d 74 d7 a8 0d 00 1e 7b 4b 3a 4a e6 0f e8 c0 71 fc 73 e7 02 4c 0d bc f4 bd d1 1d 39 6b ba 70 46 4a 13 e9 4a f8 3d f3 e1 09 59 54 7b c9 55 fb 41 2d a3 76 52 11 e1 f3 dc 77 6c aa 53 37 6e ca 3a ec be c3 aa b7 3b 31 d5 6c b6 52 9c 80 98 bc c9 e0 28 18 e2 0b f7 f8 a0 3a fd 17 04 50 9e ce 79 bd 9f 39 f1 ea 69 ec 47 97 2e 83 0f b5 ca 95 de 95 a1 e6 04 22 d5 ee be 52 79 54 a1 e7 bf 8a 86 f6 46 6d 0d 9f 16 95 1a 4c f7 a0 46 92 59 5c 13 52 f2 54 9e 5a fb 4e bf d7 7a 37 95 01 44 e4 c0 26 87 4c 65 3e 40 7d 7d 23 07 44 01 f4 84 ff d0 8f 7a 1f a0 52 10 d1 f4 f0 d5 ce 79 70 29 32 e2 ca be 70 1f df ad 6b 4b b7 11 01 f4 4b ad 66 6a 11 13 0f e2 ee 82 9e 4d 02 9d c9 1c dd 67 16 db b9 06 18 86 ed c1 ba 94 21 02 03 01 00 01 a3 52 30 50 30 0e 06 03 55 1d 0f 01 01 ff 04 04 03 02 05 a0 30 1d 06 03 55 1d 25 04 16 30 14 06 08 2b 06 01 05 05 07 03 02 06 08 2b 06 01 05 05 07 03 01 30 1f 06 03 55 1d 23 04 18 30 16 80 14 89 4f de 5b cc 69 e2 52 cf 3e a3 00 df b1 97 b8 1d e1 c1 46 30 0d 06 09 2a 86 48 86 f7 0d 01 01 0b 05 00 03 82 01 01 00 59 16 45 a6 9a 2e 37 79 e4 f6 dd 27 1a ba 1c 0b fd 6c d7 55 99 b5 e7 c3 6e 53 3e ff 36 59 08 43 24 c9 e7 a5 04 07 9d 39 e0 d4 29 87 ff e3 eb dd 09 c1 cf 1d 91 44 55 87 0b 57 1d d1 9b df 1d 24 f8 bb 9a 11 fe 80 fd 59 2b a0 39 8c de 11 e2 65 1e 61 8c e5 98 fa 96 e5 37 2e ef 3d 24 8a fd e1 74 63 eb bf ab b8 e4 d1 ab 50 2a 54 ec 00 64 e9 2f 78 19 66 0d 3f 27 cf 20 9e 66 7f ce 5a e2 e4 ac 99 c7 c9 38 18 f8 b2 51 07 22 df ed 97 f3 2e 3e 93 49 d4 c6 6c 9e a6 39 6d 74 44 62 a0 6b 42 c6 d5 ba 68 8e ac 3a 01 7b dd fc 8e 2c fc ad 27 cb 69 d3 cc dc a2 80 41 44 65 d3 ae 34 8c e0 f3 4a b2 fb 9c 61 83 71 31 2b 19 10 41 64 1c 23 7f 11 a5 d6 5c 84 4f 04 04 84 99 38 71 2b 95 9e d6 85 bc 5c 5d d6 45 ed 19 90 94 73 40 29 26 dc b4 0e 34 69 a1 59 41 e8 e2 cc a8 4b b6 08 46 36 a0 00 00 0f 00 01 04 08 04 01 00 5c bb 24 c0 40 93 32 da a9 20 bb ab bd b9 bd 50 17 0b e4 9c fb e0 a4 10 7f ca 6f fb 10 68 e6 5f 96 9e 6d e7 d4 f9 e5 60 38 d6 7c 69 c0 31 40 3a 7a 7c 0b cc 86 83 e6 57 21 a0 c7 2c c6 63 40 19 ad 1d 3a d2 65 a8 12 61 5b a3 63 80 37 20 84 f5 da ec 7e 63 d3 f4 93 3f 27 22 74 19 a6 11 03 46 44 dc db c7 be 3e 74 ff ac 47 3f aa ad de 8c 2f c6 5f 32 65 77 3e 7e 62 de 33 86 1f a7 05 d1 9c 50 6e 89 6c 8d 82 f5 bc f3 5f ec e2 59 b7 15 38 11 5e 9c 8c fb a6 2e 49 bb 84 74 f5 85 87 b1 1b 8a e3 17 c6 33 e9 c7 6c 79 1d 46 62 84 ad 9c 4f f7 35 a6 d2 e9 63 b5 9b bc a4 40 a3 07 09 1a 1b 4e 46 bc c7 a2 f9 fb 2f 1c 89 8e cb 19 91 8b e4 12 1d 7e 8e d0 4c d5 0c 9a 59 e9 87 98 01 07 bb bf 29 9c 23 2e 7f db e1 0a 4c fd ae 5c 89 1c 96 af df f9 4b 54 cc d2 bc 19 d3 cd aa 66 44 85 9c 14 00 00 30 7e 30 ee cc b6 b2 3b e6 c6 ca 36 39 92 e8 42 da 87 7e e6 47 15 ae 7f c0 cf 87 f9 e5 03 21 82 b5 bb 48 d1 e3 3f 99 79 05 5a 16 0c 8d bb b1 56 9c")
# digest = hashes.Hash(hashes.SHA384())
# digest.update(test_client_hello)
# digest.update(test_server_hello)
# digest.update(test_extension_1)
# digest.update(test_extension_2)
# digest.update(test_extension_3)
# digest.update(test_extension_4)
# digest.update(test_extension_all)
# test_hash = digest.finalize()
# print_bytes_as_hex(test_hash)


## -------- Potential bug in application_key_hash generation --------------


01 00 00 97 03 03 ae a2 33 02 24 f0 63 33 6d 3f 0d 6f e4 cc 45 12 77 10 96 85 ab 47 f0 e4 93 8f bc 30 45 dc 08 d4 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 20 0d f7 6f c7 2e bd 94 cc 6e 25 cd 36 26 e0 f0 96 4b d0 7b e3 3a 4c 99 5b 41 21 51 f6 21 37 10 00 2b 00 03 02 03 04
02 00 00 56 03 03 06 f3 47 79 8d 67 a4 67 cd a1 ba 0c c1 5b eb 3e 91 31 71 9e 5f e9 89 de 84 55 04 a4 b2 bb 33 80 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 48 fa 2d d4 05 e0 7d 21 2a cb f2 8f 08 8a 66 36 a7 19 38 75 ec 04 f3 c4 f7 4a 47 8a 4c ea 7e 5a 00 2b 00 02 03 04
08 00 00 06 00 04 00 00 00 00 0b 00 09 f8 00 00 09 f4 00 03 c4 30 82 03 c0 30 82 03 66 a0 03 02 01 02 02 10 12 c6 55 b2 e7 0b ce 7c 13 1a 7c e5 a2 e0 e7 00 30 0a 06 08 2a 86 48 ce 3d 04 03 02 30 3b 31 0b 30 09 06 03 55 04 06 13 02 55 53 31 1e 30 1c 06 03 55 04 0a 1

# 9b. Application key generation ✅
Same procedure with the handshake keys

In [800]:
# TestValues. Uncomment this and compare with https://tls13.xargs.org/#server-handshake-keys-calc
# handshake_secret = bytes.fromhex("bdbbe8757494bef20de932598294ea65b5e6bf6dc5c02a960a2de2eaa9b07c929078d2caa0936231c38d1725f179d299")
# application_context = bytes.fromhex("fa6800169a6baac19159524fa7b9721b41be3c9db6f3f93fa5ff7e3db3ece204d2b456c51046e40ec5312c55a86126f5")
# client_secret = bytes.fromhex("db89d2d6df0e84fed74a2288f8fd4d0959f790ff23946cdf4c26d85e51bebd42ae184501972f8d30c4a3e4a3693d0ef0")

zeros = bytes([0] * 32)
derived_secret = hkdf_expand_label(
    secret = handshake_secret,
    label="derived",
    context=empty_hash,
    length=48
)
master_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(),
    length=48,
    salt=derived_secret,  # derived_secret is used as salt
    info=None
)._extract(b'\x00' * 48)


client_app_secret = hkdf_expand_label(
    secret = master_secret,
    label="c ap traffic",
    context = application_context,
    length = 48
)

server_app_secret = hkdf_expand_label(
    secret = master_secret,
    label = "s ap traffic",
    context = application_context,
    length = 48
)


client_app_key = hkdf_expand_label(
    secret = client_app_secret,
    label = "key",
    context = b"",
    length = 32
)

server_app_key = hkdf_expand_label(
    secret = server_app_secret,
    label = "key",
    context = b"",
    length = 32
)

client_app_iv = hkdf_expand_label(
    secret = client_app_secret,
    label = "iv",
    context = b"",
    length = 12
)

server_app_iv = hkdf_expand_label(
    secret = server_app_secret,
    label = "iv",
    context = b"",
    length = 12
)

print("client secret: " + client_secret.hex())
print("server secret: " + server_secret.hex())
print("client app key: " + client_app_key.hex())
print("client app iv: "+ client_app_iv.hex())
print("server app key: " + server_app_key.hex())
print("server app iv: " + server_app_iv.hex())

client secret: a3c5b66a2198943058e9062fc479e09f03248f838f31bbc866c5090a77da01c8a0d4ede4bb1e566df75d40ed0ef02761
server secret: f55cacb1e93b11d5fc127f8783db01c79d076c4930a70828881ee26e3ab031fde9cd2f5304137a8fb56533bb7738e0e6
client app key: 9978c5c79444c97ad0a15e8e139bfccffde394825fc29f3e2185ed4334f6a41e
client app iv: 84b605ce3eb15290052e6edb
server app key: c22b817a69d43638d154b4a0c34bef4a82d7da9d5bf5caf9c9506fed7c16d5bc
server app iv: 0a2b735de7d88610e1f912a4


# 10. Generate and send the finished message 🟨
The last step would be to generate the Finished message and send it to the server. After that, we're finished with the handshake, and can start sending over HTTP requests

In [801]:
def calculate_verify_data(client_handshake_secret: bytes, transcript_hash: bytes) -> bytes:
    # First derive the finished key using HKDF-Expand-Label
    finished_key = hkdf_expand_label(
        secret=client_handshake_secret, 
        label="finished", 
        context=b"", 
        length=48  # SHA-384 output size
    )
    
    # Calculate the transcript hash using SHA-384
    
    # Create HMAC of the transcript hash using finished_key
    h = HMAC(finished_key, hashes.SHA384())
    h.update(transcript_hash)
    return h.finalize()

def create_client_finished_message(client_handshake_key: bytes, client_handshake_iv: bytes, client_handshake_secret: bytes, 
                                   transcript_hash: bytes) -> bytes:
    # Get the verify_data using the function above
    verify_data = calculate_verify_data(client_handshake_secret, transcript_hash)
    
    # Create the Finished message structure
    msg = bytes([0x14, 0x00, 0x00]) + len(verify_data).to_bytes(1, 'big')  # Type and length
    msg += verify_data                      # The HMAC
    msg += bytes([0x16])                    # Handshake message type
    
    # Record layer header as AAD
    print_bytes_as_hex(msg)
    msg_len = len(msg) + 16  # AES-GCM adds a 16-byte authentication tag
    additional = bytes([0x17, 0x03, 0x03]) + msg_len.to_bytes(2, 'big')
    
    # Encrypt using AES-GCM
    aesgcm = AESGCM(client_handshake_key)
    encrypted = aesgcm.encrypt(client_handshake_iv, msg, additional)
    
    # Return the full TLS record
    return additional + encrypted


finished_message = create_client_finished_message(
    client_handshake_key=client_handshake_key,
    client_handshake_iv=client_handshake_iv,
    client_handshake_secret= client_secret,
    transcript_hash = application_context
    )
print_bytes_as_hex(finished_message)


14 00 00 30 8b ab f7 60 e7 f3 38 b0 57 ab 16 35 5b e6 4d 94 1d 19 4d dd 18 ba 54 8e c6 11 b5 4b a9 8e 01 d0 a7 fe c4 5c df 39 d6 e2 55 a0 42 a2 1d 9d 9c d0 16
17 03 03 00 45 36 a3 5d 3c b7 65 bd 57 0a 30 94 e0 4c ab b7 e2 1d 1b de d1 7c 17 d7 c2 1c 9e 07 89 27 9e 7c 6a 4f 31 d8 d0 ea 7e 8f bb 07 64 68 c9 52 ec 2b 45 ff 13 08 3a 09 f2 7c 9f 4e d6 f4 08 f4 d5 96 47 4f 11 96 64 27


In [802]:
s.sendall(finished_message)
print(f"Sent Client Finished message ({len(finished_message)} bytes)")

# Receive response after Finished
response = receive_all_data(s)
print(f"Received response after Finished: {len(response)} bytes")
print_bytes_as_hex(response)


Sent Client Finished message (74 bytes)


Received response after Finished: 0 bytes



# 11. Send an HTTP request
We'll send a simple get request to test our connection

In [803]:
def send_data(s: socket.socket, client_app_key: bytes, client_app_iv: bytes, client_seq: int = 0) -> bytes:
    data = f"GET / HTTP/1.1\r\nHost: www.cloudflare.com\r\n\r\n".encode('ascii')
    # Prepare and encrypt outgoing data
    data = data + bytes([0x17])
    length_bytes = struct.pack("!H", len(data) + 16)
    additional = bytes([0x17, 0x03, 0x03]) + length_bytes
    
    # Create nonce by XORing IV with sequence number
    client_nonce = create_nonce(client_app_iv, client_seq)
    
    # Encrypt outgoing data
    aesgcm = AESGCM(client_app_key)
    encrypted = aesgcm.encrypt(client_nonce, data, additional)
    msg = additional + encrypted
    
    # Send the message
    s.sendall(msg)
    print(f"Sent message ({len(data)} bytes)")
    
    # Receive response
    response = receive_all_data(s)
    return response
    

def create_nonce(iv: bytes, seq_num: int) -> bytes:
    # Create a nonce by XORing the IV with the sequence number
    seq_bytes = seq_num.to_bytes(12, byteorder='big')
    return bytes(a ^ b for a, b in zip(iv, seq_bytes))

def decrypt_tls_record(key, base_iv, record, seq_num=0):
    # Split the record into header (AAD) and ciphertext
    header = record[:5]
    ciphertext = record[5:]
    
    # Create the nonce using your existing function
    nonce = create_nonce(base_iv, seq_num)
    
    # Initialize AES-GCM with the provided key
    aesgcm = AESGCM(key)
    
    # Decrypt the ciphertext
    try:
        plaintext = aesgcm.decrypt(nonce, ciphertext, header)
        # The last byte is the actual content type
        content_type = plaintext[-1]
        actual_data = plaintext[:-1]
        return content_type, actual_data
    except Exception as e:
        raise ValueError(f"Decryption failed: {e}")

def decrypt_response(response, server_app_key, server_app_iv):
    decrypted_data = []
    offset = 0
    seq_num = 0
    
    while offset < len(response):
        # Make sure we have at least a complete TLS record header
        if offset + 5 > len(response):
            break
        
        # Parse record length
        record_length = int.from_bytes(response[offset+3:offset+5], byteorder='big')
        
        # Make sure we have the complete record
        if offset + 5 + record_length > len(response):
            break
        
        # Extract the current record
        current_record = response[offset:offset+5+record_length]
        
        try:
            # Decrypt the record using our function
            content_type, data = decrypt_tls_record(server_app_key, server_app_iv, current_record, seq_num)
            decrypted_data.append((content_type, data))
            
            print(f"Decrypted record {seq_num}: type={content_type}, length={len(data)}")
            if content_type == 23:  # Application Data
                print(f"Sample: {data[:50].decode('utf-8', errors='replace')}...")
        except Exception as e:
            print(f"Failed to decrypt record {seq_num}: {e}")
        
        # Move to the next record
        offset += 5 + record_length
        seq_num += 1
    
    return decrypted_data

response = send_data(s, client_app_key, client_app_iv)
decrypted_records = decrypt_response(response, server_app_key, server_app_iv)

html_content = b''
for content_type, data in decrypted_records:
    if content_type == 23:  # Application Data
        html_content += data

if html_content:
    # Process HTML content
    text = html_content.decode('utf-8', errors='replace')
    # Extract body after headers
    header_end = text.find('\r\n\r\n')
    if header_end != -1:
        html_body = text[header_end + 4:]
        with open('cloudflare.html', 'w', encoding='utf-8') as f:
            f.write(html_body)
        print("HTML content saved to cloudflare.html")





Sent message (45 bytes)
Decrypted record 0: type=23, length=1265
Sample: HTTP/1.1 200 OK
Date: Fri, 14 Mar 2025 00:23:31 G...
Decrypted record 1: type=23, length=1369
Sample: 6000
<!DOCTYPE html><html lang="en-us"><head><scr...
Decrypted record 2: type=23, length=1369
Sample: == 'www.cloudflare-cn.com') {
      if (localStora...
Decrypted record 3: type=23, length=1369
Sample: hans-cn, switch to zh-cn
            const redirec...
Decrypted record 4: type=23, length=1369
Sample: ><meta property="og:url" content="https://www.clou...
Decrypted record 5: type=23, length=1369
Sample: e" hreflang="id-id" href="https://www.cloudflare.c...
Decrypted record 6: type=23, length=1369
Sample: y_interaction",page_path:`${location.pathname}${lo...
Decrypted record 7: type=23, length=1369
Sample: r("DOMContentLoaded",o);document.addEventListener(...
Decrypted record 8: type=23, length=1369
Sample: odeURIComponent(v[0].trim())] = decodeURIComponent...
Decrypted record 9: type=23, length=1369
Sample: et